In [ ]:
from __future__ import print_function, division
import os
import pandas as pd # For csv
import numpy as np

import requests
import json

### Data set-up

Import drive, so that read and write access can be used with processed files, speeding up workflow.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Downloading data from Disprot website
- Disprot API had no clear way to download the full file of annotated disordered sequences.
- Custom function uses URL from inspecting Disprots source code to download the TSV file.
- Can change the release date and constraints on the documented sequences.
- NB: TSV format and consensus false must remain to work with the rest of the processing steps.
- Project has been done using the following:
 - year = '2022_06'
 - ambiguous = 'true'
 - obsolete = 'false'
 - format = 'tsv'
 - consensus = 'false'

#### Creating a pandas dataframe containing accession IDs and disordered region locations.
- This is used to generate labels for our disordered sequences.

In [ ]:
def get_disprot_dataset():
  year = '2022_06'
  ambiguous = 'true'
  obsolete = 'false'
  format = 'tsv'
  # The rest of the processing only works if consensus is false. Assumes region file format.
  consensus = 'false'

  disprot_link = "https://disprot.org/api/search?release="+year+"&show_ambiguous="+ambiguous+"&show_obsolete="+obsolete+"&format="+format+"&namespace=all&get_consensus="+consensus
  with open('/content/drive/My Drive/Colab Notebooks/diss_files/downloaded_disprot_data.tsv', 'w') as outfile:
    outfile.write( requests.get(disprot_link).text )

  # Data from DisProt TSV - https://disprot.org/download
  data_disprot = pd.read_csv('/content/drive/My Drive/Colab Notebooks/diss_files/downloaded_disprot_data.tsv', sep='\t')
  return data_disprot

def create_dataframe_of_idrs(data_disprot):
  # Dictionary for disordered region data from DisProt
  disorder_start_and_end = {}
  for i, acc in enumerate(data_disprot['acc']):
    s = data_disprot['start'][i]
    e = data_disprot['end'][i]
    arr = disorder_start_and_end.get((str(acc)), [])
    if (s, e) not in arr:
      disorder_start_and_end[str(acc)] = arr + [(s, e)]
      
  # Create new table for important DisProt data
  data = {'acc': disorder_start_and_end.keys(), 'disordered_regions': disorder_start_and_end.values()}
  pandas_data = pd.DataFrame.from_dict(data)
  return pandas_data


In [ ]:
data_disprot = get_disprot_dataset()
pandas_data = create_dataframe_of_idrs(data_disprot)

In [ ]:
pandas_data

,acc,disordered_regions
0,P03265,"[(294, 334), (454, 464)]"
1,P49913,"[(134, 170)]"
2,P03045,"[(1, 107), (1, 22), (34, 47), (1, 36)]"
3,P00004,"[(1, 104), (2, 105)]"
4,P27695,"[(1, 42), (1, 36), (32, 43), (2, 40)]"
...,...,...
2414,A0A5P2U9X4,"[(350, 525), (460, 521), (417, 426), (450, 525)]"
2415,P40939,"[(637, 647)]"
2416,Q6CSX2,"[(562, 831)]"
2417,Q8IYT8,"[(168, 177)]"


#### Downloading the full protein sequences.
- Downloads these full sequences from UniProt.
 - If an empty string is returned this means the protein has been deprecated and we will not include this in our dataset.
- Creates a dataframe

The proteins in this dataframe are preprocessed to get their full sequences from UniProt.

In [ ]:
def preprocess_sequences(pandas_data):
  protSeqDict = {}
  for row in range(len(pandas_data)):
    acc = pandas_data['acc'].loc[row]

    url = f'https://www.uniprot.org/uniprotkb/{str(acc)}.fasta'
    uniprot_fasta = requests.get(url).text
    # Gets the sequence as a string of amino acids
    protein_sequence = uniprot_fasta.split('\n')[1:]
    protein_sequence = ''.join(protein_sequence)

    if protein_sequence == '':
      continue

    protSeqDict[acc] = protein_sequence
  return protSeqDict

protein_sequences_n_ids = preprocess_sequences(pandas_data)

In [ ]:
# Save preprocessed data
def write_sequences():
  with open('/content/drive/My Drive/Colab Notebooks/diss_files/sequence_data.json', 'w') as outfile:
    json.dump(protein_sequences_n_ids, outfile)

# Quick access to preprocessed data, instead of downloading it each time Notebook is opened.
def read_sequences():
  with open('/content/drive/My Drive/Colab Notebooks/diss_files/sequence_data.json', 'r') as infile:
    return json.load(infile)

# Commented out write sequences, so this file is not accidentally overwritten before preprocessing is ran
#write_sequences()
protein_sequences_n_ids = read_sequences()

#### Removing protein data that is incompatible with my solution.
- Removing deprecated entries from dataset. This empty string broke when compared to a label the length of the deprecated sequence.
- Removing ambiguous sequences from dataset. My solution only handles the 20 known amino acid codes.

In [ ]:
def cleaning_pandas_data():

  # Removing deprecated entries
  clean_pandas_data = pandas_data
  non_deprecated_acc = protein_sequences_n_ids.keys()
  for acc in pandas_data['acc']:
    if acc in non_deprecated_acc:
      continue
    else:
      index_to_drop = clean_pandas_data[clean_pandas_data['acc'] == acc].index.tolist()[0]
      clean_pandas_data = clean_pandas_data.drop(index_to_drop)

  # Removing ambiguous sequences
  for acc in clean_pandas_data['acc']:
    seq = protein_sequences_n_ids.get(acc)
    # These letter codes are used when an amino acid is ambiguous
    if 'X' in seq or 'U' in seq or 'Z' in seq:
      index_to_drop = clean_pandas_data[clean_pandas_data['acc'] == acc].index.tolist()[0]
      clean_pandas_data = clean_pandas_data.drop(index_to_drop)

  return clean_pandas_data.reset_index().drop(columns=['index'])

def write_cleaned_pandas_data(pandas_df):
  pandas_df.to_json('/content/drive/My Drive/Colab Notebooks/diss_files/idr_pandas_table.json')

def read_cleaned_pandas_data():
  return pd.read_json('/content/drive/My Drive/Colab Notebooks/diss_files/idr_pandas_table.json')

In [ ]:
cleaned_pandas_data = cleaning_pandas_data()
#write_cleaned_pandas_data(cleaned_pandas_data)

In [ ]:
fully_clean_pandas_data = read_cleaned_pandas_data()

Creating One-hot map

In [ ]:
def make_empty_channels(seq):
  seq_channels = {
    'A' : np.zeros(len(seq)),
    'C' : np.zeros(len(seq)),
    'D' : np.zeros(len(seq)),
    'E' : np.zeros(len(seq)),
    'F' : np.zeros(len(seq)),
    'G' : np.zeros(len(seq)),
    'H' : np.zeros(len(seq)),
    'I' : np.zeros(len(seq)),
    'K' : np.zeros(len(seq)),
    'L' : np.zeros(len(seq)),
    'M' : np.zeros(len(seq)),
    'N' : np.zeros(len(seq)),
    'P' : np.zeros(len(seq)),
    'Q' : np.zeros(len(seq)),
    'R' : np.zeros(len(seq)),
    'S' : np.zeros(len(seq)),
    'T' : np.zeros(len(seq)),
    'V' : np.zeros(len(seq)),
    'W' : np.zeros(len(seq)),
    'Y' : np.zeros(len(seq))
  }
  return seq_channels

def make_channels(seq):
  # Makes 20 empty channels
  channeled_seq = make_empty_channels(seq)
  # Loop over each amino acid in the sequence - 
  # for its position add a 1 to the letter identifier channel
  for i, char in enumerate(seq):
    # Updates array due to arrays being like pointers
    channeled_seq.get(char)[i] = 1

  return channeled_seq

In [ ]:
def get_onehot_encoding():
  onehot_map = {}
  for acc in fully_clean_pandas_data['acc']:
    channeled_seq = make_channels(protein_sequences_n_ids.get(acc))
    onehot_seq = np.array(list(channeled_seq.values()))
    onehot_map[acc] = onehot_seq

  return onehot_map

onehot_map = get_onehot_encoding()

In [ ]:
# https://stackoverflow.com/questions/26646362/numpy-array-is-not-json-serializable
from json import JSONEncoder
class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

# Save preprocessed one-hot sequences
def write_onehot(onehot_map):
  with open('/content/drive/My Drive/Colab Notebooks/diss_files/onehot_data.json', 'w') as outfile:
    json.dump(onehot_map, outfile, cls=NumpyArrayEncoder)


# Quick access to preprocessed one-hot sequences, instead of recreating the map each time Notebook is opened.
def read_onehot():
  with open('/content/drive/My Drive/Colab Notebooks/diss_files/onehot_data.json', 'r') as infile:
    onehot_map = json.load(infile)
    # https://stackoverflow.com/questions/5010536/python-perform-an-operation-on-each-dictionary-value
    onehot_map.update((acc, np.array(onehot)) for acc, onehot in onehot_map.items())
    return onehot_map

# Commented out write onehot, so this file is not accidentally overwritten before processing is ran
#write_onehot(onehot_map)
onehot_map = read_onehot()

#### Creating PSSM map
- PSSMs generated by MMSeqs2.
- Read in the PSSM and lookup table files.
- Write out a dictionary of Accession IDs : PSSMs.

In [ ]:
def pssm_ids_to_accs(id_pssm_map, acc_series):
  acc_pssm_map = {}
  for k, v in id_pssm_map.items():
    acc = acc_series[k]
    # This check must be made due to DISPROT containing the latter accession ID, but the PSSM generated with MMSeqs2 produces the former.
    # NB: They are the same protein and searching for the latter protein redirects to the former on UniProt.
    if acc == 'A0A6L8PPD0':
      acc_pssm_map['A0A0F7RL08'] = v
    else:
      acc_pssm_map[acc] = v

  return acc_pssm_map


def pssm_parser(fname, lookup):
  pssm_file = fname
  seq_ids = []
  pssms = []
  
  pssm = []
  with open(pssm_file, 'r') as f:
    for line in f:
      # Start of a new sequence PSSM
      if line.startswith("Query profile of sequence"):
        # Checks if a PSSM has been read in before this sequence
        if len(pssm) > 0:
          pssms.append(np.array(pssm).T)

        # Get sequence ID and create a new empty data structure to read in this PSSM.
        seq_id = line.strip().split()[-1]
        seq_ids.append(int(seq_id))
        pssm = []
        
      elif line.startswith("Pos"):
        continue
      # An amino acids PSSM representation. Parse this into a 20 length vector.
      else:
        values = line.strip().split()
        pssm.append([int(v) for v in values[2:]])
    
    # Add the final PSSM to the PSSMs list as there are no more new sequences
    pssms.append(np.array(pssm).T)

  seq_ids = np.array(seq_ids)    
  id_pssm_map = dict(zip(seq_ids, pssms))
  
  # Get lookup table generated by MMSeqs2. This will let us pair the integer sequence IDs to accession IDs.
  data = pd.read_csv(lookup, sep='\t', header=None)
  data.columns=['Query Seq', 'ACC', 'IND']
  # Get the accession ID series. The index of an accession ID will correlate to the integer sequence ID from the PSSM.
  acc_series = data['ACC']

  # Getting mapping to acc:PSSM from integer id:PSSM
  acc_pssm_map = pssm_ids_to_accs(id_pssm_map, acc_series)
  return acc_pssm_map

pssms_fname = '/content/drive/My Drive/Colab Notebooks/diss_files/pssms'
pssms_lookup = '/content/drive/My Drive/Colab Notebooks/diss_files/ALL.lookup'
all_pssms = pssm_parser(pssms_fname, pssms_lookup)

In [ ]:
# Save preprocessed pssms
def write_pssms(pssm_map):
  with open('/content/drive/My Drive/Colab Notebooks/diss_files/pssm_data.json', 'w') as outfile:
    json.dump(pssm_map, outfile, cls=NumpyArrayEncoder)

# Quick access to preprocessed pssms, instead of recreating the map each time Notebook is opened.
def read_pssms():
  with open('/content/drive/My Drive/Colab Notebooks/diss_files/pssm_data.json', 'r') as infile:
    pssm_map = json.load(infile)
    # https://stackoverflow.com/questions/5010536/python-perform-an-operation-on-each-dictionary-value
    pssm_map.update((acc, np.array(pssm)) for acc, pssm in pssm_map.items())
    return pssm_map

# Commented out write pssms, so this file is not accidentally overwritten before processing is ran
#write_pssms(all_pssms)
pssm_map = read_pssms()

#### Splitting the dataset
- Creating a separate train, validation and test set.
- Using a seed for reproducible results.

In [ ]:
randomly_sampled_pandas_data = fully_clean_pandas_data.sample(frac = 1, random_state=117).reset_index()

train_len = round(len(randomly_sampled_pandas_data) * (60/100))
train_pandas_data = randomly_sampled_pandas_data[0:train_len]

valid_len = round((len(randomly_sampled_pandas_data) * (20/100)))
valid_pandas_data = randomly_sampled_pandas_data[train_len:(train_len+valid_len)]
# Reset index helps clean up index column after slicing.
valid_pandas_data = valid_pandas_data.reset_index(drop=True)

test_len = len(randomly_sampled_pandas_data) - (train_len + valid_len)
test_pandas_data = randomly_sampled_pandas_data[(train_len+valid_len):]
test_pandas_data = test_pandas_data.reset_index(drop=True)

#### Handling homologous sequences
- Aim is to prevent the leakage of data between protein sequences that occur from a similar evolutionary background.

First we get the FASTA format of the sequences in each separate dataset, so they can be used in a homology search clustering algorithm.

In [ ]:
training_accs = train_pandas_data['acc'].tolist()
validation_accs = valid_pandas_data['acc'].tolist()
test_accs = test_pandas_data['acc'].tolist()

In [ ]:
def write_fasta(acc_list, filename):
  FILE_PATH = '/content/drive/My Drive/Colab Notebooks/training_data_files/'+filename
  for i, acc in enumerate(acc_list):
    url = f'https://www.uniprot.org/uniprotkb/{str(acc)}.fasta'
    uniprot_fasta = requests.get(url).text

    with open(FILE_PATH, 'a+') as outfile:
      outfile.write(uniprot_fasta + '\n')

    if i % 200 == 0:
      print("Sequence ", i)

# Commented out to prevent overwriting
#write_fasta(training_accs, 'training_sequences.fasta')
#write_fasta(validation_accs, 'validation_sequences.fasta')
#write_fasta(test_accs, 'test_sequences.fasta')

MMSeqs2 clustering, plus command line filtering to prepend the dataset name to the accession code, gives us the following TSV that we can continue to preprocess.

In [ ]:
mmseqs_homo_clusters = pd.read_csv('/content/drive/My Drive/Colab Notebooks/diss_files/clusterRes2_cluster.tsv', sep='\t', header=None)
mmseqs_homo_clusters.columns=['Homo seq', 'Unique seq']
mmseqs_homo_clusters

,Homo seq,Unique seq
0,TRAIN_Q9KMA5,TRAIN_Q9KMA5
1,TRAIN_Q53654,TRAIN_Q53654
2,TRAIN_P53330,TRAIN_P53330
3,TRAIN_P12272,TRAIN_P12272
4,TRAIN_P12497,TRAIN_P12497
...,...,...
2404,TEST_P12355,TEST_P12355
2405,TEST_Q9MAB9,TEST_Q9MAB9
2406,TEST_O80358,TEST_O80358
2407,TEST_Q9HAU5,TEST_Q9HAU5


In [ ]:
def identify_homolgous_sequences():
  col1 = mmseqs_homo_clusters['Homo seq']
  col2 = mmseqs_homo_clusters['Unique seq']
  seeping_sequences = 0
  diff_seqs = []
  col_1_diff = []
  col_2_diff = []

  for i, val in enumerate(col1):
    # If the homologous sequence is itself, continue as this is not a problem
    if val == col2[i]:
      continue

    # If the homologous sequence is in the same dataset, continue as this is not a problem.
    if val[0:5] == "TRAIN" and col2[i][0:5] == "TRAIN":
      continue
    elif val[0:3] == "VAL" and col2[i][0:3] == "VAL":
      continue
    elif val[0:4] == "TEST" and col2[i][0:4] == "TEST":
      continue
    
    diff_seqs.append((val, col2[i]))
    col_1_diff.append(val)
    col_2_diff.append(col2[i])
    seeping_sequences += 1

  print("There are", seeping_sequences, "seeping sequences.")

  homo_seqs = {'Homo seq': col_1_diff, 'Unique seq': col_2_diff}
  homo_seqs_pd = pd.DataFrame.from_dict(homo_seqs)
  return homo_seqs_pd

In [ ]:
homo_seqs_pd = identify_homolgous_sequences()

There are 286 seeping sequences.


In [ ]:
# Function creates a map so we can move all homologous sequences into a dataset with sequences they are homologous to.
# Prevents data leakage between datasets.
def moving_dataset(pandas_data):
  moved_dataset_map = {}
  for i, val in enumerate(pandas_data['Homo seq']):
    if val[0:5] == "TRAIN":
      over_writing_seq = pandas_data['Unique seq'][i]
      if over_writing_seq[0:3] == "VAL":
        moved_dataset_map[over_writing_seq] = "TRAIN" + over_writing_seq[3:]
      else:
        moved_dataset_map[over_writing_seq] = "TRAIN" + over_writing_seq[4:]

    elif val[0:3] == "VAL":
      over_writing_seq = pandas_data['Unique seq'][i]
      if over_writing_seq[0:5] == "TRAIN":
        moved_dataset_map[over_writing_seq] = "VAL" + over_writing_seq[5:]
      else:
        moved_dataset_map[over_writing_seq] = "VAL" + over_writing_seq[4:]

    elif val[0:4] == "TEST":
      over_writing_seq = pandas_data['Unique seq'][i]
      if over_writing_seq[0:5] == "TRAIN":
        moved_dataset_map[over_writing_seq] = "TEST" + over_writing_seq[5:]
      else:
        moved_dataset_map[over_writing_seq] = "TEST" + over_writing_seq[3:]

  return moved_dataset_map

moved_dataset_map = moving_dataset(homo_seqs_pd)

In [ ]:
# Cleans acc labels and moves sequences between datasets.
def organise_sequences():
  train_accs = []
  validation_accs = []
  test_accs = []

  transfer_seqs = moved_dataset_map.keys()
  for val in mmseqs_homo_clusters['Unique seq']:
    # Check if sequence needs to be moved dataset
    if val in transfer_seqs:
      acc = moved_dataset_map.get(val)
    else:
      acc = val

    # Remove randomly split dataset identification
    if acc[0:5] == "TRAIN":
      train_accs.append(acc[6:])
    elif acc[0:3] == "VAL":
      if acc[4:] == 'A0A6L8PPD0':
        new_acc = 'A0A0F7RL08'
        validation_accs.append(new_acc)
        continue
      validation_accs.append(acc[4:])
    elif acc[0:4] == "TEST":
      test_accs.append(acc[5:])

  return train_accs, validation_accs, test_accs

train_accs, validation_accs, test_accs = organise_sequences()
dataset_split = {'Train': train_accs, 'Validation': validation_accs, 'Test': test_accs}

In [ ]:
# Verify the split of our datasets is still appropriate
print("Train new percentage ", round(len(train_accs) / 2409 * 100))
print("Valid new percentage ", round(len(validation_accs) / 2409 * 100))
print("Test new percentage ", round(len(test_accs) / 2409 * 100))

Train new percentage  62
Valid new percentage  19
Test new percentage  19


In [ ]:
# Save preprocessing of homologous sequence solution
def write_homo_solution(split):
  with open('/content/drive/My Drive/Colab Notebooks/diss_files/homo_solution.json', 'w') as outfile:
    json.dump(split, outfile)

# Quick access to preprocessed dataset split, instead of doing this each time Notebook is opened.
def read_homo_solution():
  with open('/content/drive/My Drive/Colab Notebooks/diss_files/homo_solution.json', 'r') as infile:
    return json.load(infile)


# Commented out write homo solution, so this file is not accidentally overwritten before processing is ran
#write_homo_solution(dataset_split)
non_leaking_dataset_split = read_homo_solution()